In [1]:
import os

import gym
import numpy as np
import pandas as pd
import seaborn as sns

from ray import init, rllib, tune, shutdown

In [2]:
from attack_simulator.agents import DEFENDERS
from attack_simulator.alphazero_env import AttackSimulationAlphaZeroEnv
from attack_simulator.env import AttackSimulationEnv
from attack_simulator.graph import AttackGraph, SIZES

In [3]:
class AgentPolicy(rllib.policy.Policy):
    def __init__(self, observation_space, action_space, config):
        super().__init__(observation_space, action_space, config)
        agent_config = dict(
            input_dim=observation_space.shape[0],
            num_actions=action_space.n,
            random_seed=config["seed"],
            attack_graph=config["env_config"]["attack_graph"],
        )
        self._agent = DEFENDERS[config["agent_type"]](agent_config)

    def compute_actions(self, observations, *args, **kwargs):
        # FIXME: use a `numpy` array as a temporary workaround for
        #        https://github.com/ray-project/ray/issues/10100
        return np.array([self._agent.act(obs) for obs in observations]), [], {}

    def get_weights(self):
        return {}

    def set_weights(self, weights):
        pass


def template_agent(agent_type):
    default_config = rllib.agents.trainer.with_common_config(dict(agent_type=agent_type))
    return rllib.agents.trainer_template.build_trainer(
        name=agent_type,
        default_policy=AgentPolicy,
        default_config=default_config,
    )

In [4]:
# 'contrib/AlphaZero' does NOT appear to work without its custom dense model
from ray.rllib.contrib.alpha_zero.models.custom_torch_models import DenseModel

rllib.models.ModelCatalog.register_custom_model("alpha_zero_dense_model", DenseModel)

In [5]:
from ray.util.client import worker

worker.INITIAL_TIMEOUT_SEC = worker.MAX_TIMEOUT_SEC = 1


def ray_init():
    if os.path.isdir("/var/run/secrets/kubernetes.io") or os.path.exists(
        os.path.expanduser("~/ray_bootstrap_config.yaml")
    ):
        # inside k8s pod or a cluster managed by Ray's autoscaler
        context = init(address="auto")
    else:
        ray_client_server = "host.docker.internal" if os.path.exists("/.dockerenv") else "127.0.0.1"
        try:
            context = init(address=f"ray://{ray_client_server}:10001")
        except ConnectionError:
            # listen on all interfaces inside a container for port-forwarding to work
            dashboard_host = "0.0.0.0" if os.path.exists("/.dockerenv") else "127.0.0.1"
            context = init(num_cpus=4, dashboard_host=dashboard_host)
    print("\x1b[33;1m", context, "\x1b[m")
    return context

In [6]:
agent_types = ["contrib/AlphaZero", "R2D2", "rule-based", "random"]
graphs = [AttackGraph(dict(graph_size=size)) for size in SIZES]
seeds = [0, 1, 2, 3, 6, 7, 11, 28, 42, 1337]
iterations = 10
rollouts = 10

common_config = dict(
    # log_level='DEBUG',
    framework="torch",
    env=AttackSimulationEnv,
    env_config=dict(attack_graph=tune.grid_search(graphs)),
    seed=tune.grid_search(seeds),
    # common evaluation settings
    evaluation_num_workers=1,
    evaluation_config=dict(
        explore=False,
        # workaround for a bug in RLLib (https://github.com/ray-project/ray/issues/17921)
        replay_sequence_length=-1,
    ),
    evaluation_num_episodes=rollouts,
)
train_and_eval_config = dict(
    common_config,
    num_workers=4,
    # evaluation at the end
    evaluation_interval=iterations,
)
eval_only_config = dict(
    common_config,
    # evaluation ONLY: avoid MultiGPU optimizer, set all relevant sizes to 0
    simple_optimizer=True,
    num_workers=0,
    train_batch_size=0,
    rollout_fragment_length=0,
    timesteps_per_iteration=0,
    # evaluation at the end
    evaluation_interval=1,
)

AttackGraph(en2720.yaml[tiny], 2 services, 7 attack steps)
AttackGraph(en2720.yaml[small], 5 services, 27 attack steps)
AttackGraph(en2720.yaml[medium-small], 9 services, 44 attack steps)
AttackGraph(en2720.yaml[medium], 13 services, 56 attack steps)
AttackGraph(en2720.yaml[large], 16 services, 62 attack steps)
AttackGraph(en2720.yaml[extra-large], 16 services, 65 attack steps)
AttackGraph(en2720.yaml[full], 18 services, 78 attack steps)


In [7]:
rename = {
    "config.env_config.agent_type": "Agent",
    "config.env_config.attack_graph": "graph",
    "evaluation.hist_stats.episode_reward": "returns",
    "evaluation.hist_stats.episode_lengths": "lengths",
}


def postprocess(results_df):
    df = results_df[rename.keys()].rename(columns=rename)
    df.dropna(inplace=True)  # remove `NaN` evaluation results from failed trials
    df["Graph size"] = df["graph"].apply(lambda g: g.num_attacks)
    del df["graph"]
    df["tuple"] = df.apply(lambda t: list(zip(t.returns, t.lengths)), axis="columns")
    del df["returns"]
    del df["lengths"]
    df = df.explode("tuple", ignore_index=True)
    df[["Returns", "Episode lengths"]] = df["tuple"].tolist()
    del df["tuple"]
    return df

In [8]:
def generate(savename):
    ray_init()

    frames = []
    for agent_type in agent_types:
        if agent_type in DEFENDERS:
            agent = template_agent(agent_type)
            config = dict(eval_only_config)
            stop = dict(training_iteration=0)
        else:
            agent = agent_type
            if agent_type == "contrib/AlphaZero":
                config = dict(
                    train_and_eval_config,
                    env=AttackSimulationAlphaZeroEnv,
                    model=dict(custom_model="alpha_zero_dense_model"),
                    rollout_fragment_length=32,
                    train_batch_size=640,
                    buffer_size=512,
                )
            else:
                config = dict(train_and_eval_config, model=dict(use_lstm=True))

            stop = dict(training_iteration=iterations)  # TODO: additional stopping criteria?

        config["env_config"].update(agent_type=agent_type)
        results = tune.run(
            agent,
            config=config,
            stop=stop,
            max_failures=3,
            queue_trials=True,
            raise_on_failed_trial=False,
            progress_reporter=tune.JupyterNotebookReporter(overwrite=True),
        )
        frames.append(results.results_df)

    shutdown()
    results_df = pd.concat(frames, ignore_index=True)
    df = postprocess(results_df)
    df.to_csv(savename)
    return df

In [9]:
savename = "returns-tune-eval.csv"

df = generate(savename) if not os.path.exists(savename) else pd.read_csv(savename, index_col=0)

Trial name,status,loc,env_config/attack_graph,seed
contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00000,RUNNING,,"AttackGraph(en2720.yaml[tiny], 2 services, 7 attack steps)",0
contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00001,PENDING,,"AttackGraph(en2720.yaml[small], 5 services, 27 attack steps)",0
contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00002,PENDING,,"AttackGraph(en2720.yaml[medium-small], 9 services, 44 attack steps)",0
contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00003,PENDING,,"AttackGraph(en2720.yaml[medium], 13 services, 56 attack steps)",0
contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00004,PENDING,,"AttackGraph(en2720.yaml[large], 16 services, 62 attack steps)",0
contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00005,PENDING,,"AttackGraph(en2720.yaml[extra-large], 16 services, 65 attack steps)",0
contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00006,PENDING,,"AttackGraph(en2720.yaml[full], 18 services, 78 attack steps)",0
contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00007,PENDING,,"AttackGraph(en2720.yaml[tiny], 2 services, 7 attack steps)",1
contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00008,PENDING,,"AttackGraph(en2720.yaml[small], 5 services, 27 attack steps)",1
contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00009,PENDING,,"AttackGraph(en2720.yaml[medium-small], 9 services, 44 attack steps)",1


(pid=827) [2021-11-16 16:26:57,099 C 827 827] core_worker.cc:194:  Check failed: core_worker_process The core worker process is not initialized yet or already shutdown.
(pid=827) *** StackTrace Information ***
(pid=827)     ray::SpdLogMessage::Flush()
(pid=827)     ray::RayLog::~RayLog()
(pid=827)     ray::core::CoreWorkerProcess::EnsureInitialized()
(pid=827)     ray::core::CoreWorkerProcess::GetCoreWorker()
(pid=827)     __pyx_pw_3ray_7_raylet_10CoreWorker_73remove_actor_handle_reference()
(pid=827) 
(pid=934) 2021-11-16 16:26:59,423	WARNING deprecation.py:38 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
(pid=934) 2021-11-16 16:26:59,424	INFO trainer.py:770 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=141, ip=10.84.0.194) 2021-11-16 16:27:02,132	WARNING rollout_worker.py:1536 -- Validating sub-env at vector index=0 ... (NOT OK)
(pid=141, ip=10.84.0.194) 

Traceback (most recent call last):
  File "/Users/mehes/Library/Caches/pypoetry/virtualenvs/attack-simulator-5suhPhn0-py3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/gr/408vn4xd0vv2pt_0h6q9fzbr0000gn/T/ipykernel_13919/2800115950.py", line 3, in <module>
    df = generate(savename) if not os.path.exists(savename) else pd.read_csv(savename, index_col=0)
  File "/var/folders/gr/408vn4xd0vv2pt_0h6q9fzbr0000gn/T/ipykernel_13919/1859403590.py", line 27, in generate
    results = tune.run(
  File "/Users/mehes/Library/Caches/pypoetry/virtualenvs/attack-simulator-5suhPhn0-py3.9/lib/python3.9/site-packages/ray/tune/tune.py", line 353, in run
    while ray.wait([remote_future], timeout=0.2)[1]:
  File "/Users/mehes/Library/Caches/pypoetry/virtualenvs/attack-simulator-5suhPhn0-py3.9/lib/python3.9/site-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return get

(pid=181, ip=10.84.0.130) 2021-11-16 16:27:04,141	WARNING rollout_worker.py:1536 -- Validating sub-env at vector index=0 ... (NOT OK)
(pid=181, ip=10.84.0.130) 2021-11-16 16:27:04,142	ERROR worker.py:425 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=181, ip=10.84.0.130)
(pid=181, ip=10.84.0.130)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 460, in __init__
(pid=181, ip=10.84.0.130)     _validate_env(self.env, env_context=self.env_context)
(pid=181, ip=10.84.0.130)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1537, in _validate_env
(pid=181, ip=10.84.0.130)     raise EnvError(
(pid=181, ip=10.84.0.130) ray.rllib.utils.error.EnvError: Env's `observation_space` Dict(action_mask:Box([0 0 0], [1 1 1

TypeError: object of type 'NoneType' has no len()

In [ ]:
sns.set(style="darkgrid", rc={"figure.figsize": (12, 8)})

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Returns", hue="Agent", ci="sd")
g.legend(title="Agent", loc="upper left")
g.set_title("Returns vs Size (random attacker)")

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Episode lengths", hue="Agent", ci="sd")
g.legend(title="Agent", loc="upper left")
g.set_title("Episode lengths vs Size (random attacker)")

In [ ]:
# pd.set_option("display.max_columns", 32)
df.groupby("Agent").describe()

(pid=636) 2021-11-16 16:27:04,165	ERROR trial_runner.py:924 -- Trial contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00000: Error processing event.
(pid=636) Traceback (most recent call last):
(pid=636)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/lib/python3.9/site-packages/ray/tune/trial_runner.py", line 890, in _process_trial
(pid=636)     results = self.trial_executor.fetch_result(trial)
(pid=636)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/lib/python3.9/site-packages/ray/tune/ray_trial_executor.py", line 788, in fetch_result
(pid=636)     result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
(pid=636)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/lib/python3.9/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
(pid=636)     return func(*args, **kwargs)
(pid=636)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/lib/python3.9/site-packages/

(run pid=636) Result for contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00000:
(run pid=636)   trial_id: 97b74_00000
(run pid=636)   


(pid=1037) [2021-11-16 16:27:12,519 C 1037 1037] core_worker.cc:194:  Check failed: core_worker_process The core worker process is not initialized yet or already shutdown.
(pid=1037) *** StackTrace Information ***
(pid=1037)     ray::SpdLogMessage::Flush()
(pid=1037)     ray::RayLog::~RayLog()
(pid=1037)     ray::core::CoreWorkerProcess::EnsureInitialized()
(pid=1037)     ray::core::CoreWorkerProcess::GetCoreWorker()
(pid=1037)     __pyx_pw_3ray_7_raylet_10CoreWorker_73remove_actor_handle_reference()
(pid=1037) 
(pid=1082) 2021-11-16 16:27:14,956	WARNING deprecation.py:38 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
(pid=1082) 2021-11-16 16:27:14,956	INFO trainer.py:770 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=183, ip=10.84.0.194) 2021-11-16 16:27:17,666	WARNING rollout_worker.py:1536 -- Validating sub-env at vector index=0 ... (NOT OK)
(pid=183, ip=1

(run pid=636) Result for contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00001:
(run pid=636)   trial_id: 97b74_00001
(run pid=636)   


(pid=1215) [2021-11-16 16:27:43,633 C 1215 1215] core_worker.cc:194:  Check failed: core_worker_process The core worker process is not initialized yet or already shutdown.
(pid=1215) *** StackTrace Information ***
(pid=1215)     ray::SpdLogMessage::Flush()
(pid=1215)     ray::RayLog::~RayLog()
(pid=1215)     ray::core::CoreWorkerProcess::EnsureInitialized()
(pid=1215)     ray::core::CoreWorkerProcess::GetCoreWorker()
(pid=1215)     __pyx_pw_3ray_7_raylet_10CoreWorker_73remove_actor_handle_reference()
(pid=1215) 
(pid=1260) 2021-11-16 16:27:46,066	WARNING deprecation.py:38 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
(pid=1260) 2021-11-16 16:27:46,066	INFO trainer.py:770 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=267, ip=10.84.0.194) 2021-11-16 16:27:48,785	WARNING rollout_worker.py:1536 -- Validating sub-env at vector index=0 ... (NOT OK)
(pid=267, ip=1

...

(pid=1749) [2021-11-16 16:29:19,026 C 1749 1749] core_worker.cc:194:  Check failed: core_worker_process The core worker process is not initialized yet or already shutdown.
(pid=1749) *** StackTrace Information ***
(pid=1749)     ray::SpdLogMessage::Flush()
(pid=1749)     ray::RayLog::~RayLog()
(pid=1749)     ray::core::CoreWorkerProcess::EnsureInitialized()
(pid=1749)     ray::core::CoreWorkerProcess::GetCoreWorker()
(pid=1749)     __pyx_pw_3ray_7_raylet_10CoreWorker_73remove_actor_handle_reference()
(pid=1749) 
(pid=1795) 2021-11-16 16:29:21,404	WARNING deprecation.py:38 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
(pid=1795) 2021-11-16 16:29:21,405	INFO trainer.py:770 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=519, ip=10.84.0.194) 2021-11-16 16:29:24,146	WARNING rollout_worker.py:1536 -- Validating sub-env at vector index=0 ... (NOT OK)
(pid=519, ip=1

(run pid=636) Result for contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00005:
(run pid=636)   trial_id: 97b74_00005
(run pid=636)   


(pid=636) 2021-11-16 16:29:50,148	WARNING ray_trial_executor.py:768 -- Over the last 60 seconds, the Tune event loop has been backlogged processing new results. Consider increasing your period of result reporting to improve performance.
(pid=636) 2021-11-16 16:29:50,148	ERROR trial_runner.py:924 -- Trial contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00005: Error processing event.
(pid=636) Traceback (most recent call last):
(pid=636)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/lib/python3.9/site-packages/ray/tune/trial_runner.py", line 890, in _process_trial
(pid=636)     results = self.trial_executor.fetch_result(trial)
(pid=636)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/lib/python3.9/site-packages/ray/tune/ray_trial_executor.py", line 788, in fetch_result
(pid=636)     result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
(pid=636)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/li

(run pid=636) Result for contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00006:
(run pid=636)   trial_id: 97b74_00006
(run pid=636)   


(pid=636) 2021-11-16 16:30:22,319	ERROR trial_runner.py:924 -- Trial contrib_AlphaZero_AttackSimulationAlphaZeroEnv_97b74_00006: Error processing event.
(pid=636) Traceback (most recent call last):
(pid=636)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/lib/python3.9/site-packages/ray/tune/trial_runner.py", line 890, in _process_trial
(pid=636)     results = self.trial_executor.fetch_result(trial)
(pid=636)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/lib/python3.9/site-packages/ray/tune/ray_trial_executor.py", line 788, in fetch_result
(pid=636)     result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
(pid=636)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/lib/python3.9/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
(pid=636)     return func(*args, **kwargs)
(pid=636)   File "/root/.cache/pypoetry/virtualenvs/attack-simulator-_8mgyPy8-py3.9/lib/python3.9/site-packages/